

#  图异常检测（Graph Anomaly Detection, GAD）

---

## 1. 异常检测任务定义

### 1.1 任务目标

> 图异常检测旨在从图结构数据中识别“异常”元素（节点、边或子图），这些元素在结构或属性上**偏离了主流模式**。

### 1.2 异常类别

| 异常类型                   | 说明             | 例子              |
| ---------------------- | -------------- | --------------- |
| 节点异常（Node Anomaly）     | 节点的特征或连接模式显著偏离 | 僵尸账户、恶意用户       |
| 边异常（Edge Anomaly）      | 边的存在不合理，或边属性异常 | 欺诈交易、社交网络中的虚假连接 |
| 子图异常（Subgraph Anomaly） | 子图整体行为或连接方式异常  | 虚假新闻传播网络、异常社团   |

### 1.3 输入输出格式

* 输入：图 $G = (V, E, X)$，其中 $V$ 为节点集，$E$ 为边集，$X \in \mathbb{R}^{|V| \times d}$ 为节点属性矩阵。
* 输出：每个节点/边/子图的异常评分或标签（0：正常，1：异常）

### 1.4 面临的挑战

* 图结构复杂：节点和边之间存在高阶依赖
* 异常稀少且多样：异常数量少，类型多变，难以建模
* 标签稀缺：异常样本标注成本高，限制监督学习方法
* 属性结构异构：节点属性可能与图结构不一致（异配）




## 2. 基于分类的异常检测方法

### 2.1 基本思路

将图异常检测建模为**监督或半监督分类问题**，训练模型区分“正常”和“异常”节点/边：

$$
\mathcal{L}_{cls} = \text{CrossEntropy}(f_\theta(X, A), Y)
$$

其中：

* $f_\theta$：图神经网络提取器（如 GCN、GAT）
* $Y$：已知的标签（正常/异常）

### 2.2 模型流程

1. 使用图神经网络提取节点表示 $Z = f_\theta(X, A)$
2. 用 MLP 对 $Z$ 进行二分类
3. 根据预测概率判断是否异常

### 2.3 代表方法

| 模型                    | 机制                    | 特点        |
| --------------------- | --------------------- | --------- |
| **CARE-GNN (KDD’20)** | 异常传播感知 GNN，利用多跳信息和边类型 | 能处理异常稀缺问题 |
| **SEGA (CIKM’21)**    | 半监督 GNN，融合节点属性与结构信息   | 稀疏标注下表现出色 |

### 2.4 优点与局限

* 能显式利用标签，准确率高
* 标签依赖性强，不适合无标签或异常泛化场景



## 3. 基于自编码器的异常检测方法

### 3.1 核心思想

> 通过图自编码器（Graph Autoencoder, GAE）重构图结构或节点属性，**重构误差大者视为异常**。

### 3.2 重构机制

#### 结构重构：

* 编码器：$Z = f_\theta(X, A)$
* 解码器：$\hat{A} = \sigma(ZZ^\top)$
* 异常评分（结构误差）：

  $$
  s^{(i)} = \| A_i - \hat{A}_i \|_2
  $$

#### 属性重构：

* 解码器：$\hat{X} = g_\phi(Z)$
* 异常评分（属性误差）：

  $$
  s^{(i)} = \| X_i - \hat{X}_i \|_2
  $$

#### 联合重构（属性 + 结构）：

* 总损失函数：

  $$
  \mathcal{L} = \lambda_1 \| A - \hat{A} \|_F^2 + \lambda_2 \| X - \hat{X} \|_F^2
  $$

### 3.3 典型模型

| 模型                     | 特点                    |
| ---------------------- | --------------------- |
| **DOMINANT (ICDM’19)** | 联合重构结构+属性，异常评分基于误差累加  |
| **AnomalyDAE**         | 加入去噪机制，增强鲁棒性          |
| **GCNAE**              | 用 GCN 替代传统编码器，提升结构敏感性 |

### 3.4 优点与缺点

* 完全无监督，适用于无标签数据
* 可解释性强：误差来源明确
* 容易过拟合正常结构，难识别新颖异常



## 4. 基于对比学习的图异常检测方法

### 4.1 核心思想

> 利用对比学习框架构建正负样本对，训练图神经网络提取判别性嵌入，异常节点无法在对比中建立“相似性”。

### 4.2 典型流程

1. 构造正样本对：

   * 同一节点的不同视图（结构扰动/属性 mask）
   * 相邻节点或同社区节点
2. 构造负样本对：

   * 随机采样其他节点
3. 使用 InfoNCE 损失优化节点表示：

   $$
   \mathcal{L}_{CL} = - \log \frac{\exp(\text{sim}(z_i, z_j)/\tau)}{\sum_k \exp(\text{sim}(z_i, z_k)/\tau)}
   $$
4. 异常分数：节点在多个视图之间表示不一致 → 异常

---

### 4.3 代表方法

| 模型                   | 机制                 | 优点         |
| -------------------- | ------------------ | ---------- |
| **CoLA (KDD’21)**    | 自监督结构-属性对比，构造图对比目标 | 无需标签，鲁棒性高  |
| **ANEMONE (WWW’22)** | 多视图对比 + 噪声建模       | 可处理异配图中的异常 |
| **GADCL**            | 子图级别对比学习           | 能捕捉局部结构异常  |

---

### 4.4 优势总结

* 无需标签 → 适配大规模场景
* 鲁棒性强 → 能抗扰动/结构噪声
* 对比样本质量影响大，调参复杂



## 5. 其他异常检测方法

### 5.1 基于生成模型的方法

> 使用图生成模型（如 VAE、GAN）重建图结构/属性，通过样本与生成分布的差异判断异常。

#### 示例模型：

* **GraphVAE**：潜在变量建模图生成概率
* **GraphGANDetect**：利用判别器判定样本是否异常

---

### 5.2 子图模式识别方法

* 提取**频繁子图模式**，若某节点或边不属于主流模式 → 异常
* 如 FSMiner、Pattern-Based GAD

---

### 5.3 流图/动态图异常检测

> 图随时间变化，需在线检测异常

* 方法：滑动窗口、时间感知 GCN、事件驱动模型
* 示例：

  * **STAND**：动态图流上节点异常检测
  * **DyGAD**：动态图上用 RNN/GNN 检测突变节点

---

### 5.4 基于注意力机制的检测

* 用 GAT/GAM 模型提取节点影响力
* 注意力显著偏离 → 潜在异常节点

---

## 总结对比表

| 方法类别 | 监督性     | 适用场景         | 优点          | 代表方法            |
| ---- | ------- | ------------ | ----------- | --------------- |
| 分类方法 | 有监督/半监督 | 小规模有标签图      | 高准确率        | CARE-GNN, SEGA  |
| 自编码器 | 无监督     | 属性结构联合建模     | 易解释，结构感知    | DOMINANT, GCNAE |
| 对比学习 | 自监督     | 无标签、大规模图     | 鲁棒性高，适配多样异常 | CoLA, ANEMONE   |
| 生成模型 | 无监督     | 潜在结构建模       | 可估计概率、判别度量强 | GraphGANDetect  |
| 子图挖掘 | 无监督     | 图模式挖掘、异常社群检测 | 高语义可解释性     | FSMiner         |
| 动态检测 | 无监督     | 实时异常检测       | 时间敏感，适合变化图  | STAND, DyGAD    |

